In [1]:
import os
import pathlib

project_root = pathlib.Path.cwd()

cache_dir = project_root / "models_cache"
cache_dir.mkdir(exist_ok=True)
os.environ['HF_HOME'] = str(cache_dir)
print(f"La variable de entorno HF_HOME se ha establecido en: {os.environ['HF_HOME']}")


La variable de entorno HF_HOME se ha establecido en: /Users/deimagjas/machinelearning/models_cache


In [2]:
import json
from typing import Dict, List, Tuple, Union

import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import load, generate
from mlx_lm.tuner import TrainingArgs, linear_to_lora_layers, train

In [3]:
!python3 -m pip show mlx_lm

Name: mlx-lm
Version: 0.28.0
Summary: LLMs with MLX and the Hugging Face Hub
Home-page: https://github.com/ml-explore/mlx-lm
Author: MLX Contributors
Author-email: mlx@group.apple.com
License: MIT
Location: /Users/deimagjas/machinelearning/.venv/lib/python3.12/site-packages
Requires: jinja2, mlx, numpy, protobuf, pyyaml, transformers
Required-by: 


In [4]:
from huggingface_hub import login

login()

## Carga de modelo desde HF

La razón por la que el código funciona con google/gemma-3-270m-it pero no con google/gemma-3-270m se debe a la diferencia entre los
  dos tipos de modelos:

   1. `google/gemma-3-270m-it`: El sufijo "-it" significa "Instruction Tuned" (ajustado para instrucciones). Este modelo ha sido
      específicamente entrenado para entender y seguir instrucciones en un formato de chat o de pregunta-respuesta. Su tokenizador
      incluye una "plantilla de chat" (chat template) que formatea la entrada de manera que el modelo la entienda.

   2. `google/gemma-3-270m`: Este es el modelo base. Es un modelo de lenguaje pre-entrenado que es bueno para predecir la siguiente
      palabra en un texto, pero no ha sido ajustado para seguir instrucciones o para conversar. Su tokenizador no tiene una plantilla de
      chat predeterminada.

In [5]:
model_path = "google/gemma-3-270m-it"
model, tokenizer = load(model_path)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
prompt = "generate an SQL query to find all users who registered in the last 30 days"
messages = [{"role": "user", "content": prompt}]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate(model, tokenizer, prompt=prompt, verbose=True)

```sql
SELECT *
FROM Users
WHERE registration_date >= DATE('now', '-30 days');
```

**Explanation:**

* **`SELECT *`**: This selects all columns from the `Users` table.
* **`FROM Users`**: This specifies the table from which to retrieve data.
* **`WHERE registration_date >= DATE('now', '-30 days')`**: This is the filtering condition.
    * `registration_date`:  This is the column name that contains the date of the user's registration.
    * `DATE('now', '-30 days')`: This calculates the date 30 days ago from the current date.
    * `>=`: This ensures that we only include users whose registration date is greater than or equal to 30 days ago.

**Important Considerations:**

* **Data Type of `registration_date`:**  The `registration_date` column should be a date or datetime data type.  If it's stored as a string, you might need to convert it to a date type using `STR_TO_DATE('now', '-30 days')` before using it in the `WHERE` clause.
* **Database
Prompt: 26 tokens, 25.028 tokens-per-sec
Ge

# Creando Adaptador

In [7]:
adapter_path = "adapters_gemma3"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

# Lora config
Aquí se ajustan los hyperparámetros para el entrenamiento

In [8]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 8,
        "scale": 20.0,
        "dropout": 0.0,
    },
}

In [9]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)

In [10]:
training_args = TrainingArgs(
    adapter_file=adapter_file_path,
    iters=200,
    steps_per_eval=50,
    grad_checkpoint=True,
)

# Parameters and adapter
La razón por la que ves 163,840 parámetros entrenables en lugar de los 270 millones del modelo completo es porque no estás
  re-entrenando el modelo entero. Estás utilizando una técnica de ajuste fino de alta eficiencia de parámetros (PEFT) llamada LoRA 
  (Low-Rank Adaptation).

  Así es como funciona en tu notebook:

   1. Congelar el modelo base: En la celda con el id: a3b86f5c, la primera línea es model.freeze(). Esto "congela" todos los 270 millones
      de parámetros del modelo Gemma, haciendo que no sean entrenables.

   2. Inyectar adaptadores LoRA: La siguiente línea, linear_to_lora_layers(...), añade pequeños "adaptadores" o capas de bajo rango a
      ciertas partes del modelo (en tu caso, a 8 capas, según se define en lora_config).

   3. Entrenar solo los adaptadores: Solo se entrenan los parámetros de estos nuevos y pequeños adaptadores. El número 163,840 es la suma
      de todos los parámetros de estas nuevas capas LoRA que se han añadido.

  En resumen:

   * 270 Millones: Es el tamaño total del modelo base, que permanece sin cambios.
   * 163,840: Es el número de parámetros nuevos y adicionales que estás entrenando. Estos parámetros son los que "aprenden" la nueva
     tarea (en este caso, generar consultas SQL) y adaptan el conocimiento del modelo original.

  Esta es la gran ventaja de LoRA: te permite especializar un modelo enorme en una tarea específica de forma muy rápida y con muchos
  menos recursos computacionales, ya que solo necesitas entrenar una fracción minúscula (<0.1%) de los parámetros totales.

In [11]:
model.freeze()
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])
num_train_params = sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
print(f"Number of trainable parameters: {num_train_params}")
model.train()

Number of trainable parameters: 163840


Model(
  (model): Gemma3Model(
    (embed_tokens): Embedding(262144, 640)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=640, output_dims=1024, bias=False)
        (k_proj): Linear(input_dims=640, output_dims=256, bias=False)
        (v_proj): Linear(input_dims=640, output_dims=256, bias=False)
        (o_proj): Linear(input_dims=1024, output_dims=640, bias=False)
        (q_norm): RMSNorm()
        (k_norm): RMSNorm()
        (rope): RoPE(256, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): Linear(input_dims=640, output_dims=2048, bias=False)
        (down_proj): Linear(input_dims=2048, output_dims=640, bias=False)
        (up_proj): Linear(input_dims=640, output_dims=2048, bias=False)
      )
      (input_layernorm): RMSNorm()
      (post_attention_layernorm): RMSNorm()
      (pre_feedforward_layernorm): RMSNorm()
      (post_feedforward_layernorm): RMSNorm()
    )
    (layers.1): TransformerBlock(
      (self_att

In [12]:
class Metrics:
    def __init__(self) -> None:
        self.train_losses: List[Tuple[int, float]] = []
        self.val_losses: List[Tuple[int, float]] = []

    def on_train_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.train_losses.append((info["iteration"], info["train_loss"]))

    def on_val_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.val_losses.append((info["iteration"], info["val_loss"]))

In [13]:
metrics = Metrics()

# load data

In [14]:
from mlx_lm.tuner.datasets import load_hf_dataset
config = { }
train_set, val_set, test_set = load_hf_dataset(
    data_id="mlx-community/wikisql",
    tokenizer=tokenizer,
    config=config,
)

In [15]:
print(f"Test set size: {len(test_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Training set size: {len(train_set)}")
print(f"test set: {test_set[:2]}")

Test set size: 100
Validation set size: 100
Training set size: 1000
test set: {'text': ["table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What is terrence ross' nationality\nA: SELECT Nationality FROM 1-10015132-16 WHERE Player = 'Terrence Ross'", "table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What clu was in toronto 1995-96\nA: SELECT School/Club Team FROM 1-10015132-16 WHERE Years in Toronto = '1995-96'"]}


In [16]:
from mlx_lm.tuner.datasets import CacheDataset

train_dataset = CacheDataset(train_set)
val_dataset = CacheDataset(val_set)

train(
    model,
    optim.Adam(learning_rate=1e-5),
    train_dataset,
    val_dataset,
    args=training_args,
    training_callback=metrics
)


Starting training..., iters: 200


Calculating loss...: 100%|██████████| 25/25 [00:04<00:00,  6.07it/s]

Iter 1: Val loss 4.059, Val took 4.128s


Iter 10: Train loss 3.635, Learning Rate 1.000e-05, It/sec 1.406, Tokens/sec 565.731, Trained Tokens 4024, Peak mem 2.052 GB
Iter 20: Train loss 3.268, Learning Rate 1.000e-05, It/sec 2.693, Tokens/sec 992.073, Trained Tokens 7708, Peak mem 2.176 GB
Iter 30: Train loss 2.874, Learning Rate 1.000e-05, It/sec 2.969, Tokens/sec 1030.744, Trained Tokens 11180, Peak mem 2.176 GB
Iter 40: Train loss 2.662, Learning Rate 1.000e-05, It/sec 2.733, Tokens/sec 975.928, Trained Tokens 14751, Peak mem 2.176 GB


Calculating loss...: 100%|██████████| 25/25 [00:03<00:00,  6.87it/s]

...

Iter 150: Val loss 2.026, Val took 4.936s


Iter 150: Train loss 2.004, Learning Rate 1.000e-05, It/sec 2.531, Tokens/sec 889.487, Trained Tokens 55213, Peak mem 2.836 GB
Iter 160: Train loss 2.016, Learning Rate 1.000e-05, It/sec 2.495, Tokens/sec 927.766, Trained Tokens 58931, Peak mem 2.836 GB
Iter 170: Train loss 1.948, Learning Rate 1.000e-05, It/sec 2.603, Tokens/sec 928.039, Trained Tokens 62496, Peak mem 2.836 GB
Iter 180: Train loss 1.979, Learning Rate 1.000e-05, It/sec 2.615, Tokens/sec 987.947, Trained Tokens 66274, Peak mem 2.836 GB
Iter 190: Train loss 2.035, Learning Rate 1.000e-05, It/sec 2.433, Tokens/sec 900.097, Trained Tokens 69974, Peak mem 2.836 GB


Calculating loss...: 100%|██████████| 25/25 [00:05<00:00,  4.47it/s]

Iter 200: Val loss 1.961, Val took 5.662s


Iter 200: Train loss 1.937, Learning Rate 1.000e-05, It/sec 2.255, Tokens/sec 842.506, Trained Tokens 73710, Peak mem 2.837 GB
Iter 200: Saved adapter weights to adapters_gemma3/adapters.safetensors and adapters_gemma3/0000200_adapters.safetensors.
Saved final weights to adapters_gemma3/adapters.safetensors.


## Fusionar modelo base con adaptador

In [17]:
! python -m mlx_lm fuse  --model ./models_cache/hub/models--google--gemma-3-270m-it/snapshots/ac82b4e820549b854eebf28ce6dedaf9fdfa17b3 --adapter-path ./adapters_gemma3 --save-path ./new_gemma3 

Python(53565) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model


In [18]:
! pip install huggingface_hub
! pip install ipywidgets

Python(53609) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(53613) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Subir modelo a HF
Utilizando el API de HF se sube el modelo a deimagjas/Phi-3.5-mini-instruct-4bit-sft

In [19]:
from huggingface_hub import create_repo, upload_folder

repo_id = "deimagjas/gemma-3-270m-it-sft"

upload_folder(
    folder_path="./new_gemma3",
    repo_id=repo_id
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...elearning/new_gemma3/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

  ...arning/new_gemma3/model.safetensors:  11%|#1        | 59.3MB /  536MB            

CommitInfo(commit_url='https://huggingface.co/deimagjas/gemma-3-270m-it-sft/commit/1ebfc09cb3ebb1142d6682040993bda05e5adc2f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1ebfc09cb3ebb1142d6682040993bda05e5adc2f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/deimagjas/gemma-3-270m-it-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='deimagjas/gemma-3-270m-it-sft'), pr_revision=None, pr_num=None)

## Test HF model

In [20]:
model_path = "deimagjas/gemma-3-270m-it-sft"
model_sft, tokenizer_sft = load(model_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

In [21]:
prompt = "generate an SQL query to find all users who registered in the last 30 days"
messages = [{"role": "user", "content": prompt}]
prompt = tokenizer_sft.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate(model_sft, tokenizer_sft, prompt=prompt, verbose=True)

```sql
SELECT user_name FROM users WHERE last_30_days = '2023-12-26'
```
<end_of_turn><end_of_turn>
<end_of_turn>
<end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn>
<end_of_turn>
<end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<end_of_turn>िक<end_of_turn>िक<end_of_turn><end_of_turn><end_of_turn>िक<e

# Conclusión

El fine tunning en este caso fallo, el modelo presenta fallas en la inferencia. ¿por qué?
Ademas consume 5X memoria mas o menos